In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point, LineString

1. Import the data located at this link. It has information on people infected with dengue at the district level for 2015 to 2021.

In [ ]:
base_dengue = pd.read_csv(r'../../_data/data_dengue_peru.csv',thousands=',', na_values=['NaN', 'nan'], dtype={'Casos': 'float'})

2. Generate ubigeo for Departments and Provinces taking the first two and four numbers. Hint: Use this code.

In [ ]:
# Create 'Ubigeo_Departamento', 'Ubigeo_Provincia' and rename 'Ubigeo' to 'Ubigeo_Distrito'
base_dengue['Ubigeo_Departamento'] = base_dengue['Ubigeo'].astype(str).str[:2]
base_dengue['Ubigeo_Provincia'] = base_dengue['Ubigeo'].astype(str).str[:4]
base_dengue = base_dengue.rename(columns={'Ubigeo':'Ubigeo_Distrito'})

In [ ]:
# Create data frames 'base_Distrito', 'base_Provincia', and 'base_Departamento' to facilitate the creation of the next plots

# Define the levels we are going to group the columns 'Casos'
nivel = ['Distrito', 'Provincia', 'Departamento']
# Dictionary to store the results
datas = {}
# Loop to create the data frames
for nivel in nivel:
    columna_ubigeo = f'Ubigeo_{nivel}'
    datas[nivel] = base_dengue.groupby([columna_ubigeo, 'Año', nivel ])['Casos'].sum(min_count=0).reset_index()
    
    exec(f"base_{nivel} = datas['{nivel}']")

In [ ]:
base_Distrito

In [ ]:
# Get geometry information
maps = gpd.read_file(r'../../_data/shape_file/DISTRITOS.shp')

In [ ]:
# Keeps geometry information for 'Distrito'
maps_DIST = maps[['IDDIST', 'geometry']]
maps_DIST = maps_DIST.rename({'IDDIST':'Ubigeo_Distrito'}, axis =1 )
maps_DIST['Ubigeo_Distrito'] = maps_DIST['Ubigeo_Distrito'].astype(str)

# Keeps geometry information for 'Provincia'
maps_PROV = maps[['IDPROV', 'geometry']]
maps_PROV = maps_PROV.rename({'IDPROV':'Ubigeo_Provincia'}, axis =1 )
maps_PROV['Ubigeo_Provincia'] = maps_PROV['Ubigeo_Provincia'].astype(str)

# Keeps geometry information for 'Departamento'
maps_DEP = maps[['IDDPTO', 'geometry']]
maps_DEP = maps_DEP.rename({'IDDPTO':'Ubigeo_Departamento'}, axis =1 )
maps_DEP['Ubigeo_Departamento'] = maps_DEP['Ubigeo_Departamento'].astype(str)

3. Use geopandas to plot the number of cases in 2021 by the district using a continuous legend. Do not forget to indicate the color of NA values. Use this shapefile.

In [ ]:
#Usamos la base filtrando el año 2021 y la renombramos

base_Distrito_2021 = base_Distrito[base_Distrito["Año"] == 2021]

In [ ]:
#Checamos que los valores en las columnas Ubigeo_Distrito son únicas para ambas bases

maps_DIST.Ubigeo_Distrito.is_unique
base_Distrito_2021.Ubigeo_Distrito.is_unique

In [ ]:
#Checamos que tipos de datos son los valores en ambas bases, con énfasis en Ubigeo_Distrito para hacer el merge

maps_DIST.dtypes
base_Distrito_2021.dtypes

In [ ]:
#Convertimos los valores de la columna Ubigeo_Distrito en la base_Distrito_2021 a string

base_Distrito_2021.loc[:, 'Ubigeo_Distrito'] = base_Distrito_2021['Ubigeo_Distrito'].astype(str)

In [ ]:
#Relizamos el merge, dejando todas las observaciones de la base maps_DIST, lo que nos genera valores NaN.

dataset = pd.merge(maps_DIST, base_Distrito_2021, how = "left", on = "Ubigeo_Distrito" )

In [ ]:
#Para poder observar los distintos valores únicos en la columna Casos se realiza una tabla de frecuencias

tabla_frecuencias_casos = dataset['Casos'].value_counts().reset_index()
tabla_frecuencias_casos.columns = ['Número de Casos', 'Frecuencia']
tabla_frecuencias_casos = tabla_frecuencias_casos.sort_values(by='Número de Casos')

tabla_frecuencias_casos

In [ ]:
!pip install --upgrade mapclassify

In [ ]:
#Se hace el mapeo considerando un colo diferente para los valores NaN

import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(20, 20))
dataset.plot(ax = ax, 
        column='Casos', 
         cmap= 'Reds', 
         figsize=(20, 20), 
         linestyle='--',
         edgecolor='black', 
         legend = True,  
         scheme = "User_Defined", 
         missing_kwds= dict(color = "#DADADB",), 
         classification_kwds = dict( bins = [ 100, 200, 300, 400, 500, 1000, 1500, 2000,  2500, 3000 ] ), 
         legend_kwds=dict(  loc='upper left',
                            bbox_to_anchor=(1.01, 1),
                            fontsize='x-large',
                            title= "Número de casos por Distrito", 
                            title_fontsize = 'x-large', 
                            frameon= False )
            )